In [1]:

################################################################################
# A GENERAL EXPLANATION

"""
ligand_interface.py

This script performs interface structure prediction (high-resolution docking)
on a ligand-protein complex. This sample script is very similar to docking.py
without interface location prediction (the low-resolution (centroid) docking
stages). The high-resolution (fullatom) docking stage consists of small
rigid-body perturbations, sidechain packing, and minimization. Without
interface location prediction, this sample script cannot perform full ligand-
protein docking. The "ligand" scoring function is optimized for
ligand-protein docking.

The "Alternate scenarios" section below provides guidelines for using additional
ligand compounds in PyRosetta. A second method,
generate_nonstandard_residue_set, is supplied here to allow this script to run
in older and newer versions of PyRosetta.

Instructions:

1) ensure that your PDB file is in the current directory
2) obtain ATP.params file from /demos/data
3) ***uncomment lines 321-322***
4) run the script:
    from commandline                        >python D120_Ligand_interface.py

    from within python/ipython              [1]: run D120_Ligand_interface.py

"""

################################################################################
# THE BASIC PROTOCOL, sample_ligand_interface

"""
This sample script is setup for usage with
    commandline arguments,
    default running within a python interpreter,
    or for import within a python interpreter,
        (exposing the sample_ligand_interface method)

The method sample_ligand_interface:
1.  creates a pose from the desired PDB file, if ligand_params are specified,
        use these when loading the pose_from_file
2.  sets up the pose FoldTree for docking
3.  creates a copy of the pose to be modified
4.  creates a ScoreFunctions for scoring ligand-protein complexes
5.  sets up the DockMCMProtocol object for fullatom docking
6.  creates a (Py)JobDistributor for managing multiple trajectories
7.  create a PyMOL_Observer for viewing intermediate output
8.  exports the original structure to PyMOL
9.  perform protein-protein docking:
        a. set necessary variables for the new trajectory
            -reset the pose structure to the input conformation
            -change the pose's PDBInfo.name, for exporting to PyMOL
        b. perform Rosetta high-resolution docking
        c. output the decoy structure
            -to PyMOL using the PyMOL_Observer.pymol.apply
            -to a PDB file using the PyJobDistributor.output_decoy

"""
import sys
import optparse    # for sorting options
sys.path.append('/home/domain/data/prog/pyrosetta/')
from rosetta import *
init(extra_options = "-constant_seed")
# normally, init() works fine
# for this sample script, we want to ease comparison by making sure all random
#    variables generated by Rosetta in this instance of PyRosetta start from a
#    constant seed
# here we provide the additional argument "-constant_seed" which sets all the
#    random variables generated by Rosetta from a constant seed (google random
#    seed for more information)
# some options can be set after initialization, please see PyRosetta.org FAQs
#    for more information
# WARNING: option '-constant_seed' is for testing only! MAKE SURE TO REMOVE IT IN PRODUCTION RUNS!!!!!

import os
os.chdir('/home/domain/data/kirill/AB/XOP/rosetta')




In [2]:
def sample_ligand_interface(pdb_filename, partners,
        ligand_params = [''], jobs = 1, job_output = 'ligand_output' ):
    """
    Performs ligand-protein docking using Rosetta fullatom docking
        (DockingHighRes) on the ligand-protein complex in  <pdb_filename>
        using the relative chain  <partners>  .
        If the ligand parameters (a .params file, see below) are not defaultly
        loaded into PyRosetta,  <ligand_params>  must supply the list of files
        including the ligand parameters.
        <jobs>  trajectories are performed with output structures named
        <job_output>_(job#).pdb.
    """
    # 1. creates a pose from the desired PDB file
    pose = Pose()
    if ligand_params[0]:    # the params list has contents
        ligand_params = Vector1(ligand_params)
        new_res_set = generate_nonstandard_residue_set(ligand_params)
        pose_from_pdb(pose, new_res_set, pdb_filename)
    else:
        pose_from_pdb(pose, pdb_filename)

    # 2. setup the docking FoldTree
    # using this method, the jump number 1 is automatically set to be the
    #    inter-body jump
    dock_jump = 1
    # the exposed method setup_foldtree takes an input pose and sets its
    #    FoldTree to have jump 1 represent the relation between the two docking
    #    partners, the jump points are the residues closest to the centers of
    #    geometry for each partner with a cutpoint at the end of the chain,
    # the second argument is a string specifying the relative chain orientation
    #    such as "A_B" of "LH_A", ONLY TWO BODY DOCKING is supported and the
    #    partners MUST have different chain IDs and be in the same pose (the
    #    same PDB), additional chains can be grouped with one of the partners,
    #    the "_" character specifies which bodies are separated
    # the third argument...is currently unsupported but must be set (it is
    #    supposed to specify which jumps are movable, to support multibody
    #    docking...but Rosetta doesn't currently)
    # the FoldTrees setup by this method are for TWO BODY docking ONLY!
    setup_foldtree(pose, partners, Vector1([dock_jump]))

    # 3. create a copy of the pose for testing
    test_pose = Pose()
    test_pose.assign(pose)

    # 4. create ScoreFunctions for centroid and fullatom docking
    scorefxn = create_score_function('ligand')

    #### global docking, a problem solved by the Rosetta DockingProtocol,
    ####    requires interface detection and refinement
    #### as with other protocols, these tasks are split into centroid (interface
    ####    detection) and high-resolution (interface refinement) methods
    #### without a centroid representation, low-resolution ligand-protein
    ####    prediction is not possible and as such, only the high-resolution
    ####    ligand-protein interface refinement is available
    #### WARNING: if you add a perturbation or randomization step, the
    ####    high-resolution stages may fail (see Changing Ligand Docking
    ####    Sampling below)
    #### a perturbation step CAN make this a global docking algorithm however
    ####    the rigid-body sampling preceding refinement requires extensive
    ####    sampling to produce accurate results and this algorithm spends most
    ####    of its effort in refinement (which may be useless for the predicted
    ####    interface)

    # 5. setup the high resolution (fullatom) docking protocol (DockMCMProtocol)
    # ...as should be obvious by now, Rosetta applications have no central
    #    standardization, the DockingProtocol object can be created and
    #    applied to perform Rosetta docking, many of its options and settings
    #    can be set using the DockingProtocol setter methods
    # there is currently no centroid representation of an arbitrary ligand in
    #    the chemical database, although you can check to see if it is already
    #    present or make your own (see "Obtaining Params Files" below), and
    #    without a centroid representation, the low-resolution docking stages
    #    are not useful for ligand docking
    docking = DockMCMProtocol()
    docking.set_scorefxn(scorefxn)

    # 6. setup the PyJobDistributor
    jd = PyJobDistributor(job_output, jobs, scorefxn)

    # 7. setup a PyMOL_Observer (optional)
    # the PyMOL_Observer object owns a PyMOL_Mover and monitors pose objects for
    #    structural changes, when changes are detected the new structure is
    #    sent to PyMOL
    # fortunately, this allows investigation of full protocols since
    #    intermediate changes are displayed, it also eliminates the need to
    #    manually apply the PyMOL_Mover during a custom protocol
    # unfortunately, this can make the output difficult to interpret (since you
    #    aren't explicitly telling it when to export) and can significantly slow
    #    down protocols since many structures are output (PyMOL can also slow
    #    down if too many structures are provided and a fast machine may
    #    generate structures too quickly for PyMOL to read, the
    #    "Buffer clean up" message
    # uncomment the line below to use PyMOL_Observer
##    AddPyMolObserver(test_pose, True)

    # 8. perform protein-protein docking
    counter = 0    # for pretty output to PyMOL
    while not jd.job_complete:
        # a. set necessary variables for this trajectory
        # -reset the test pose to original (centroid) structure
        test_pose.assign(pose)
        # -change the pose name, for pretty output to PyMOL
        counter += 1
        test_pose.pdb_info().name(job_output + '_' + str(counter))

        # b. perform docking
        docking.apply(test_pose)

        # c. output the decoy structure:
        # to PyMOL
        test_pose.pdb_info().name(job_output + '_' + str(counter) + '_fa')
        # to a PDB file
        jd.output_decoy(test_pose)



In [3]:

def generate_nonstandard_residue_set(params_list):

    res_set  = ChemicalManager.get_instance().nonconst_residue_type_set('fa_standard')
    atoms    = ChemicalManager.get_instance().atom_type_set('fa_standard')
    mm_atoms = ChemicalManager.get_instance().mm_atom_type_set('fa_standard')
    orbitals = ChemicalManager.get_instance().orbital_type_set('fa_standard')
    elements = ChemicalManager.get_instance().element_set('fa_standard')
        
    res_set.read_files(params_list, atoms, elements, mm_atoms, orbitals)

    return res_set


In [4]:
################################################################################
# INTERPRETING RESULTS

"""
The (Py)JobDistributor will output the lowest scoring pose for each trajectory
(as a PDB file), recording the score in <job_output>.fasc. Generally,
the decoy generated with the lowest score contains the best prediction
for the protein conformation. PDB files produced from docking will contain
both docking partners in their predicted conformation. When inspecting these
PDB files (or the PyMOL_Observer output) be aware that PyMOL can introduce or
predict bonds that do not exist, particularly for close atoms. This rarely
occurs when using the PyMOL_Mover.keep_history feature (since PyRosetta will
sample some conformation space that has clashes).

The PyMOL_Observer will output a series of structures directly produced by the
DockingProtocol. Unfortunately, this may include intermediate structures that
do not yield any insight into the protocol performance. A LARGE number of
structures are output to PyMOL and your machine may have difficulty
loading all of these structures. If this occurs, try changing the
PyMOL_Observer keep_history to False or running the protocol without the
PyMOL_Observer.
Interface structure prediction is useful for considering what physical
properties are important in the binding event and what conformational changes
occur. Once experienced using PyRosetta, you can easily write scripts to
investigate the Rosetta score terms and structural characteristics. There is no
general interpretation of ligand-binding results. Although Rosetta score does
not translate directly to physical meaning (it is not physical energy),
splitting the docked partners and comparing the scores (after packing or
refinement) can indicate the strength of the bonding interaction
(unfortunately, the pose manipulation tools in PyRosetta are currently
undergoing repairs though manually splitting a PDB is very easy).

"""

################################################################################
# COMMANDLINE COMPATIBILITY

# everything below is added to provide commandline usage,
#   the available options are specified below
# this method:
#    1. defines the available options
#    2. loads in the commandline or default values
#    3. calls dna_sample_ligand_interface with these values

# parser object for managing input options
# all defaults are for the example using "test_lig.pdb" with reduced
#    cycles/jobs to provide results quickly
"""parser = optparse.OptionParser()
parser.add_option('--pdb_filename', dest = 'pdb_filename',
    default = '../demos/data/test_lig.pdb',    # default example PDB
    help = 'the PDB file containing the ligand and protein to dock')
# for more information on "partners", see sample_docking step 2.
parser.add_option('--partners', dest = 'partners',
    default = 'E_X',    # default for the example test_lig.pdb
    help = 'the relative chain partners for docking')
# ligand options
parser.add_option('--ligand_params', dest = 'ligand_params' ,
    default = 'ATP.params' ,    # default for the example test_lig.pdb
    help = 'the ligand residue parameter file')
# PyJobDistributor options
parser.add_option('--jobs', dest='jobs',
    default = '1',    # default to single trajectory for speed
    help = 'the number of jobs (trajectories) to perform')
parser.add_option('--job_output', dest = 'job_output',
    default = 'ligand_output',    # if a specific output name is desired
    help = 'the name preceding all output, output PDB files and .fasc')
(options,args) = parser.parse_args()

# PDB file option
pdb_filename = options.pdb_filename
partners = options.partners
# ligand options
ligand_params = options.ligand_params.split(',')
# JobDistributor options
jobs = int(options.jobs)
job_output = options.job_output
"""
# uncomment the command line below to run this demo. Make sure you have already
#      placed the ATP.params file into (in PyRosetta main directory)
#/rosetta_database/chemical/residue_type_sets/fa_standard/residue_types

'''sample_ligand_interface(pdb_filename, partners, ligand_params,
    jobs, job_output)
'''


'sample_ligand_interface(pdb_filename, partners, ligand_params,\n    jobs, job_output)\n'

In [5]:
# PDB file option
pdb_filename = "reactive21c.pdb"
partners = "A_X"
# ligand options
ligand_params = ['/home/domain/data/kirill/AB/XOP/prepare_ligs/XOPR.params']
#ligand_params = ['']

# JobDistributor options
jobs = 8
job_output = "docking_out"

In [6]:
sample_ligand_interface(pdb_filename, partners, ligand_params,
    jobs, job_output)

In [7]:
a = DockingHighRes()